### Custom Training Container Setup

#### Local Training and Eval Tests

In [ ]:
TRAINING_TABLE = "as-dev-anze.vaimlops_custom_model.training_202404292105"

In [ ]:
!python ./training/task.py --model-dir . --mode TRAINING --bq-training-table-id $TRAINING_TABLE

In [ ]:
!python ./training/task.py --model-dir . --mode EVAL --bq-training-table-id $TRAINING_TABLE

#### Local Serving Tests

In [ ]:
# Run below in the terminal. Make sure fastapi and uvicorn are pip installed
# export AIP_STORAGE_URI=gs://as-dev-anze-vaimlops-custom-model-pipelines/training_run_id_202404292038/model
# export AIP_PREDICT_ROUTE=/predict
# export AIP_HEALTH_ROUTE=/health
# cd ./serving/app
# uvicorn main:app --reload --log-level=debug

In [ ]:
import requests

In [ ]:
# check /health route
x = requests.get("http://127.0.0.1:8000/health")
print(x.status_code)

In [ ]:
from google.cloud import bigquery
bqc = bigquery.Client()
X_te = bqc.query(query=f"SELECT * EXCEPT (data_split, label) FROM `{TRAINING_TABLE}` LIMIT 1").to_dataframe()
X_te

In [ ]:
from json import JSONEncoder
import datetime

class CustomJSONEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime.datetime):
            return obj.isoformat()
            
        if isinstance(obj, datetime.date):
            return obj.strftime("%Y-%m-%d")

data = json.dumps({
    "instances": X_te.values.tolist(),
    "parameters": {"columns": list(X_te.columns)}
}, cls=CustomJSONEncoder)
print (data)

In [ ]:
x = requests.post(
    "http://127.0.0.1:8000/predict",
    headers={'Content-Type': 'application/json'},
    data=data
)

if x.status_code == 200:
    from io import BytesIO
    print (BytesIO(x.content).read().decode())

else:
    print (x)

#### Build a training Docker image

In [ ]:
GCP_PROJECT_ID = "as-dev-anze"
GCP_REGION = "us-central1"
BQ_DATASET_ID = "vaimlops_custom_model"

In [ ]:
REPO_NAME_DOCKER = BQ_DATASET_ID.replace('_', '-') + "-custom"
GCR_DOCKER = f"{GCP_REGION}-docker.pkg.dev/{GCP_PROJECT_ID}/{REPO_NAME_DOCKER}/vai-training-container:prod"

GCR_DOCKER

In [ ]:
# create artifacts docker repository
!gcloud artifacts repositories create $REPO_NAME_DOCKER --repository-format=docker --location=$GCP_REGION  --project=$GCP_PROJECT_ID

In [ ]:
SRC_DIR = "training"

In [ ]:
%cd $SRC_DIR/
!gcloud builds submit --region={GCP_REGION} --tag={GCR_DOCKER} --suppress-logs
%cd ..

#### Build a serving/prediction Docker image

In [ ]:
GCR_DOCKER_SERVING = f"{GCP_REGION}-docker.pkg.dev/{GCP_PROJECT_ID}/{REPO_NAME_DOCKER}/vai-serving-container:prod"
GCR_DOCKER_SERVING

In [ ]:
SRC_DIR = "serving"

In [ ]:
%cd $SRC_DIR/
!gcloud builds submit --region={GCP_REGION} --tag={GCR_DOCKER_SERVING} --suppress-logs
%cd ..